# Collection setup and data load

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL)
print(WEAVIATE_KEY)

if(WEAVIATE_URL == "UPDATE_ME_WEAVIATE_URL"):
    raise Exception("Please update .env and Restart the notebook (see Restart button, next to Run All)")

## Connect to Weaviate

You need to pass in your Weaviate Cloud URL and KEY.

In [ ]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

## Create a collection with a vectorizer

* [Weaviate Docs - collection creation and configuration](https://weaviate.io/developers/weaviate/manage-data/collections)
* [Weaviate integrated embedding models](https://weaviate.io/developers/weaviate/model-providers/weaviate/embeddings)

Examples of other embedding models:
* [Cohere](https://weaviate.io/developers/weaviate/model-providers/cohere/embeddings)
* [HuggingFace 🤗](https://weaviate.io/developers/weaviate/model-providers/huggingface/embeddings)
* [Ollama (self-hosted)](https://weaviate.io/developers/weaviate/model-providers/ollama/embeddings)
* [OpenAI](https://weaviate.io/developers/weaviate/model-providers/openai/embeddings)

In [ ]:
from weaviate.classes.config import Configure

if client.collections.exists("Jeopardy"):
    client.collections.delete("Jeopardy")

# Create a collection - with Weaviate vectorizer
client.collections.create(
    name="Jeopardy",
    # TODO: add text2vec_weaviate vectorizer - with:
    # * model - Snowflake/snowflake-arctic-embed-l-v2.0
    
)

## Import data
### Sample Data

In [ ]:
import json

with open("./jeopardy_tiny.json") as file:
    data_10 = json.load(file)

print(json.dumps(data_10[0:2], indent=2))

### Insert Many

> `insert_many` is only used for inserting small batches of data - must complete within the timeout.

[Weaviate Docs - insert many](https://weaviate.io/developers/weaviate/manage-data/import)

In [ ]:
# Insert data

# TODO: get Jeopardy collection
# TODO: insert data_10

### Data preview

In [ ]:
# Show data preview
jeopardy = client.collections.get("Jeopardy")

# TODO: fetch 4 objects
# response = jeopardy

for item in response.objects:
    print(item.uuid, item.properties)

In [ ]:
# Show data preview - with vectors
response = jeopardy.query.fetch_objects(
    limit=4,
    # TODO: add include_vectors
)

for item in response.objects:
    print(item.properties)
    print(item.vector, '\n')

### Super quick query example

In [ ]:
# TODO: add near text query, search for African animals with limit 2
# response = jeopardy.query

for item in response.objects:
    print(item.properties)

## Close the client

In [ ]:
client.close()